In [37]:
import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import os
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

In [38]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)
training_data.data[0].size()

torch.Size([28, 28])

In [44]:
train_dataloader = DataLoader(training_data, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=4, shuffle=True)
next(iter(train_dataloader))[1].dtype

torch.int64

In [9]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(in_features=28*28, out_features=20),
    nn.ReLU(),
    nn.Linear(20, 10)
)
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [34]:
for epoch in range(5):
    for x_train, y_train in tqdm(train_dataloader):
        y_pred = model(x_train) # делаем предсказания
        print(, y_train)
        break
        loss = nn.CrossEntropyLoss()(y_pred, y_train) # считаем лосс
        loss.backward() # считаем градиенты обратным проходом
        optimizer.step() # обновляем параметры сети
        optimizer.zero_grad() # обнуляем посчитанные градиенты параметров
    break
    mean_val_loss = [] # сюда будем складывать средний лосс по батчам
    val_accuracy = []
    with torch.no_grad(): # мы считаем качество, поэтому мы запрещаем фреймворку считать градиенты по параметрам
        for x_val, y_val in tqdm(test_dataloader): # берем батч из вал лоадера
            y_pred = model(x_val) # делаем предсказания
            loss = nn.CrossEntropyLoss()(y_pred, y_val) # считаем лосс
            mean_val_loss.append(loss.numpy()) # добавляем в массив 
            val_accuracy.extend((torch.argmax(y_pred, dim=-1) == y_val).numpy().tolist())
    print('Epoch: {epoch}, loss: {loss}, accuracy: {accuracy}'.format(epoch=epoch, loss=np.mean(mean_val_loss), accuracy=np.mean(val_accuracy)))

  0%|          | 0/15000 [00:00<?, ?it/s]

tensor([[ -0.6011,  12.0335,  -1.3341,   4.0659,   0.8288,  -9.3607,  -1.5886,
          -1.1713,  -2.9488,  -3.7245],
        [  2.8210,  11.1362,   0.5633,   5.1048,  -0.1051, -12.7422,  -1.2816,
          -3.1058,  -3.1627,  -5.8256],
        [ -2.8955,   1.5315,   5.5903,  -0.6014,   6.3271,  -3.2246,   3.9398,
          -7.2709,  -1.1803,  -7.5263],
        [  1.0021,   3.3338,  -0.3566,   7.6992,  -1.1745,  -7.5382,  -0.6912,
          -2.3422,  -2.5256,  -2.4599]], grad_fn=<AddmmBackward0>) tensor([1, 1, 4, 3])
